In [ ]:
import os
import re
import multiprocessing
from tqdm import tqdm
from gplearn_memetic.simplification import replace_C_with_indices, replace_C_indices_with_values
from gplearn_memetic._program import BAD_PRED_VALUE
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
from pmlb import fetch_data
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

for random_state in [11284, 11964, 15795, 21575, 22118, 23654, 29802,  5390,  6265, 860]:# SRBench train_test_split random seed

    BATCH_SIZE = 1000000
    
    dataset_list = ['1027_ESL',
     '1029_LEV',
     '1030_ERA',
     '1096_FacultySalaries',
     '192_vineyard',
     '210_cloud',
     '228_elusage',
     '230_machine_cpu',
     '485_analcatdata_vehicle',
     '519_vinnie',
     '522_pm10',
     '523_analcatdata_neavote',
     '529_pollen',
     '547_no2',
     '556_analcatdata_apnea2',
     '557_analcatdata_apnea1',
     '561_cpu',
     '579_fri_c0_250_5',
     '594_fri_c2_100_5',
     '596_fri_c2_250_5',
     '597_fri_c2_500_5',
     '599_fri_c2_1000_5',
     '609_fri_c0_1000_5',
     '612_fri_c1_1000_5',
     '628_fri_c3_1000_5',
     '663_rabe_266',
     '665_sleuth_case2002',
     '678_visualizing_environmental',
     '687_sleuth_ex1605',
     '690_visualizing_galaxy',
     '706_sleuth_case1202',
     '712_chscase_geyser1',]
    
    # Load dataset once and preprocess
    for dataset_name in dataset_list:
        X, y = fetch_data(dataset_name, return_X_y=True, local_cache_dir='./pmlb_data/')
        X, X_test, y, y_test = train_test_split(
            X, y, train_size=0.75, test_size=0.25, random_state=random_state
        )
        scaler_X = StandardScaler()
        X = scaler_X.fit_transform(X)
        X_test = scaler_X.transform(X_test)
    
        scaler_y = StandardScaler()
        y = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()
        y_test = scaler_y.transform(y_test.reshape(-1, 1)).flatten()
    
        def cost(C, X, y, executable_expr_skeleton): 
            y_pred = eval(executable_expr_skeleton)
            if isinstance(y_pred, (float,int)):
                y_pred = np.repeat(y_pred, X.shape[0])
            if not np.all(np.isfinite(y_pred)):
                y_pred = np.repeat(BAD_PRED_VALUE, X.shape[0])
            return mean_squared_error(y, y_pred)
    
        # Function to process each expression (to be used with multiprocessing)
        def process_expression(expr_skeleton):
            executable_expr_skeleton_C, C_count = replace_C_with_indices(expr_skeleton)
            executable_expr_skeleton = re.sub(r'X(\d+)', r'X[:,\1]', executable_expr_skeleton_C)
    
            if C_count:  # If constants need optimization
                np.random.seed(42)
                C = (2 * np.random.rand(C_count) - 1)
                try:
                    res = minimize(cost, x0=C, args=(X, y, executable_expr_skeleton), method="BFGS")
                    C = res.x
                    nit = res.nit
                except:
                    nit = 0
                executable_expr_final = replace_C_indices_with_values(executable_expr_skeleton, C)
                optimized_program = replace_C_indices_with_values(executable_expr_skeleton_C, C)
            else:
                C = None
                nit = None
                executable_expr_final = executable_expr_skeleton
                optimized_program = executable_expr_skeleton_C
    
            try:
                answer = eval(executable_expr_final)
                if isinstance(answer, (float,int)):
                    answer = np.repeat(answer, X.shape[0])
                if not np.all(np.isfinite(answer)):
                    answer = np.repeat(BAD_PRED_VALUE, X.shape[0])
        
                mse = mean_squared_error(y, answer)
                rmse = np.sqrt(mse)
                mae = mean_absolute_error(y, answer)
                r2 = r2_score(y, answer)
                result_train = [mse, rmse, mae, r2]
            except:
                result_train = [None] * 4
            
            try:        
                answer_test = eval(executable_expr_final.replace('y','y_test').replace('X','X_test'))
                if isinstance(answer_test, (float,int)):
                    answer_test = np.repeat(answer_test, X_test.shape[0])
                if not np.all(np.isfinite(answer_test)):
                    answer_test = np.repeat(BAD_PRED_VALUE, X_test.shape[0])
        
                mse_test = mean_squared_error(y_test, answer_test)
                rmse_test = np.sqrt(mse)
                mae_test = mean_absolute_error(y_test, answer_test)
                r2_test = r2_score(y_test, answer_test)
                result_test = [mse_test, rmse_test, mae_test, r2_test]
            except:
                result_test = [None] * 4
                
            return [expr_skeleton, C, optimized_program, nit] + result_train + result_test
    
        for expr_length in range(1, 12):
            new_csv_filename = f"optimized_data/{dataset_name}_seed{random_state}_len{expr_length}.csv"
            new_feather_filename = f"optimized_data/{dataset_name}_seed{random_state}_len{expr_length}.feather"
            file_path = f"skeleton_data/len{expr_length}_features{X.shape[1]}.feather"
    
            if not os.path.exists(file_path):
                print(f"skeleton_data file for {expr_length=}, features={X.shape[1]} does not exist")
                continue
    
            if os.path.exists(new_feather_filename):
                print(f"{new_feather_filename} done")
                continue
    
            df = pd.read_feather(file_path)
            expression_list = df["Expression"].tolist()
    
            # Check if the CSV file already exists to resume processing
            processed_expressions = set()
            if os.path.exists(new_csv_filename):
                existing_df = pd.read_csv(new_csv_filename, usecols=["Expression Skeleton"])
                processed_expressions = set(existing_df["Expression Skeleton"])
                print(f"Resuming {new_csv_filename}, skipping {len(processed_expressions)} already processed expressions.")
    
            # Filter out expressions that are already done
            expressions_to_process = [expr for expr in expression_list if expr not in processed_expressions]
    
            if not expressions_to_process:
                print(f"All expressions are already processed for length {expr_length}. Skipping...")
                continue
    
            # Process in batches of BATCH_SIZE
            for i in range(0, len(expressions_to_process), BATCH_SIZE):
                batch = expressions_to_process[i:i + BATCH_SIZE]
                print(f"Processing batch {i // BATCH_SIZE + 1} of {len(expressions_to_process) // BATCH_SIZE + 1} for length {expr_length}")
    
                results_data = []
                with multiprocessing.Pool(processes=max(1, os.cpu_count()-1)) as pool:
                    for result in tqdm(
                        pool.imap_unordered(process_expression, batch),
                        total=len(batch),
                        desc=f"Batch {i // BATCH_SIZE + 1} processing",
                        unit="expr"
                    ):
                        if result:
                            results_data.append(result)
    
                # Save after processing each batch
                if results_data:
                    results_data_df = pd.DataFrame(
                        results_data,
                        columns=[
                            "Expression Skeleton", "C", "Full Expression", "Optimization Iterations",
                            "MSE Train Fitness", "RMSE Train Fitness", "MAE Train Fitness", "R2 Train Fitness",
                            "MSE Test", "RMSE Test", "MAE Test", "R2 Test",
                        ]
                    )
                    # Append to CSV
                    results_data_df.to_csv(new_csv_filename, mode='a', header=not os.path.exists(new_csv_filename), index=False)
                    print(f"Saved batch {i // BATCH_SIZE + 1} results to {new_csv_filename}")
    
            # Load CSV file
            df = pd.read_csv(new_csv_filename)
            df.drop_duplicates(subset=['Expression Skeleton'], inplace=True)
            df.sort_values(by=['Expression Skeleton'], inplace=True)
            # Save back to CSV (cleaned version)
            df.to_csv(new_csv_filename, index=False)
            df.to_feather(new_feather_filename)
            print(f"Cleaned and converted {new_csv_filename} → {new_feather_filename}")
    
            print(f"Finished processing all batches for length {expr_length}")